In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install vecstack


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


In [ ]:

trainfile = r'/gdrive/MyDrive/CIS508/Homesite/RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/MyDrive/CIS508/Homesite/RevisedHomesiteTest1.csv'
test_data = pd.read_csv(testfile)
test_data.describe 


<bound method NDFrame.describe of         CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                     13                22                4                4   
1                      4                 5                8               14   
2                      3                 3               11               18   
3                      5                 9               14               22   
4                     12                21                4                5   
...                  ...               ...              ...              ...   
173831                14                23               13               21   
173832                 7                13               10               17   
173833                12                22                4                4   
173834                 6                11                4                5   
173835                 2                 2               16               23   

     

In [ ]:
k

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,...,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
0,2,1,17,23,17,23,15,22,16,22,...,1,0,0,1,0,1,0,0,0,0
1,5,9,6,8,6,8,5,7,5,8,...,1,0,0,1,0,0,0,1,0,0
2,4,6,7,12,7,12,6,10,7,11,...,1,0,0,1,0,0,0,1,0,0
3,15,23,3,2,3,2,2,2,3,2,...,1,0,0,1,0,0,0,0,1,0
4,4,6,8,13,8,13,7,11,7,13,...,1,0,0,1,0,0,1,0,0,0


In [ ]:
test_data["GeographicField64"]

0         IL
1         NJ
2         NJ
3         TX
4         CA
          ..
173831    NJ
173832    CA
173833    CA
173834    TX
173835    NJ
Name: GeographicField64, Length: 173836, dtype: object

In [ ]:
#Training data does not have a column called GeographicField64, so drop it from test
train_data["GeographicField64"]

KeyError: ignored

In [ ]:
test_data.drop(labels = ["GeographicField64"], inplace = True, axis = 1)

In [ ]:
test_data.shape

(173836, 595)

In [ ]:
train_data.shape

(65000, 596)

In [ ]:
#Do one-hot encoding of categorical variables

categoricalFeatures = ["job",
"marital",
"education",
"default",
"housing",
"loan",
"contact",
"month",
"poutcome"]


#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([train_data,test_data], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data["y"]=combined_Data["y"].map({"yes":1,"no":0})
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)
print(combined_Data['y'])

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)
y_train=X_train["y"]
y_test=X_test["y"]
X_train1=X_train.iloc[:, :-1].copy()
X_test1=X_test.iloc[:, :-1].copy()


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)





0  0        0
   1        0
   2        0
   3        0
   4        0
           ..
1  45206    1
   45207    1
   45208    1
   45209    0
   45210    0
Name: y, Length: 49732, dtype: int64


In [ ]:
TrainCols = train_data.columns
TestCols = test_data.columns
for i in TestCols:
  if i not in TrainCols:
    print(i)

In [ ]:
Xtrain = train_data[TrainCols[0:len(TrainCols)-1]].copy()
Ytrain = train_data[['QuoteConversion_Flag']].copy()
print(Xtrain.shape)
print(Ytrain.shape)
Xtest = test_data.copy()

(65000, 595)
(65000, 1)


In [ ]:
# Since test data does not have a target class for checking, we split training itself into train and test further
X_train, X_test, y_train, y_test = train_test_split(Xtrain, Ytrain, random_state=0)
X_train1 = X_train.copy()
X_test1 = X_test.copy()

In [ ]:
# SMOTE
sm = SMOTE(random_state= 42, sampling_strategy = 0.5)
X_res, y_res = sm.fit_resample(X_train, y_train)
X_res.shape

(59395, 595)

In [ ]:
X_train.shape
# print("\n")
#X_test.shape
# print("\n")
#y_train.shape
# print("\n")
#y_test.shape

(48750, 595)

In [ ]:
y_train["QuoteConversion_Flag"].value_counts()

0    39597
1     9153
Name: QuoteConversion_Flag, dtype: int64

In [ ]:
y_res.value_counts()

QuoteConversion_Flag
0                       39597
1                       19798
dtype: int64

In [ ]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(X_res,y_res)
clf_predict=clf.predict(X_test)
print("Accuracy Score (Training) for Decision Tree:{0:6f}".format(clf.score(X_res,y_res)))
print("Confusion Matrix for Decision Tree")
print("Accuracy Score (Testing) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print(confusion_matrix(y_test,clf_predict))



Accuracy Score (Training) for Decision Tree:1.000000
Confusion Matrix for Decision Tree
Accuracy Score (Testing) for Decision Tree:0.881354
[[12148   993]
 [  935  2174]]


In [ ]:
predictions = clf.predict(test_data)
sample_submission = pd.read_csv("/gdrive/MyDrive/CIS508/Homesite/sample_submission 3 (1).csv")
sample_submission['QuoteConversion_Flag'] = predictions
sample_submission.to_csv("/gdrive/MyDrive/CIS508/Homesite/submission.csv",index=False)

In [ ]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train1, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train1,y_train)
clf_predict = clf.predict(X_test1)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("Accuracy Score (Testing) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

#get cross-validation report
clf_cv_score = cross_val_score(clf, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())


{'min_samples_split': 80, 'max_depth': 9}


NameError: ignored

In [ ]:
#Construct Random Forest Model

rfc = RandomForestClassifier()
rfc = rfc.fit(X_res, y_res)
rfc_predict=rfc.predict(X_test1)
print("Accuracy Score (Testing) for RandomForest:{0:6f}".format(rfc.score(X_test1,y_test)))
print("Accuracy Score (Training) for RandomForest:{0:6f}".format(rfc.score(X_res,y_res)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
# print('Printing the precision and recall, among other metrics')
# print(metrics.classification_report(Y_test, Y_PredNew))

Accuracy Score (Testing) for RandomForest:0.901415
Accuracy Score (Training) for RandomForest:1.000000
Confusion Matrix for Random Forest:
[[12784   357]
 [ 1245  1864]]


In [ ]:
Y_PredNew = rfc.predict(X_test1)
Y_new = rfc.predict(X_train1)

#Model Accuracy
print("Training Accuracy:", metrics.accuracy_score(y_train, Y_new))
print("Testing Accuracy:", metrics.accuracy_score(y_test,Y_PredNew))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(y_test, Y_PredNew))

Training Accuracy: 1.0
Testing Accuracy: 0.9016615384615385
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

           0       0.91      0.97      0.94     13141
           1       0.83      0.61      0.70      3109

    accuracy                           0.90     16250
   macro avg       0.87      0.79      0.82     16250
weighted avg       0.90      0.90      0.90     16250



In [ ]:
predictions = rfc.predict(test_data)
sample_submission = pd.read_csv("/gdrive/MyDrive/CIS508/Homesite/sample_submission 3 (1).csv")
sample_submission['QuoteConversion_Flag'] = predictions
sample_submission.to_csv("/gdrive/MyDrive/CIS508/Homesite/submission.csv",index=False)

In [ ]:
#Hyperparameter tuning for random forest classifier
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train1, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#Construct Random Forest with best parameters
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train1,y_train)
rfc_predict = rfc.predict(X_test1)
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))

#get cross-validation report
rfc_cv_score = cross_val_score(rfc, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'min_samples_split': 80, 'max_depth': 5}
accuracy Score (training) after hypertuning for Random Forest:0.999359
Confusion Matrix after hypertuning for Random Forest:
[[39922     0]
 [   29  5260]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39922
           1       1.00      0.99      1.00      5289

    accuracy                           1.00     45211
   macro avg       1.00      1.00      1.00     45211
weighted avg       1.00      1.00      1.00     45211

=== All AUC Scores ===
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  1.0


In [ ]:
#Construct MultiLayer Perceptron Model
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(max_iter=100)
mlp.fit(X_res, y_res)
mlp_predict=mlp.predict(X_test)
print("Accuracy Score (Testing) for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test,y_test)))
print("Confusion Matrix for MultiLayer Perceptron:")
print(confusion_matrix(y_test,mlp_predict))


Accuracy Score (Testing) for MultiLayer Perceptron:0.815569
Confusion Matrix for MultiLayer Perceptron:
[[13118    23]
 [ 2974   135]]


In [ ]:
predictions = mlp.predict(test_data)
sample_submission = pd.read_csv("/gdrive/MyDrive/CIS508/Homesite/sample_submission 3 (1).csv")
sample_submission['QuoteConversion_Flag'] = predictions
sample_submission.to_csv("/gdrive/MyDrive/CIS508/Homesite/submission.csv",index=False)

In [ ]:
#Hyperparameter tuning done for MultiLayer Perceptron classifier

#parameters = {'hidden_layer_sizes':[(10,), (20,)], 'activation':['tanh', 'relu'], 'solver':['sgd', 'adam'], 'alpha': [0.0001, 0.05], 'learning_rate':['constant', 'adaptive']}
#parameters = {'hidden_layer_sizes':[(10,5), (20,5)], 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive']}
parameters = {'hidden_layer_sizes':[(10,5,3), (20,7,3)], 'activation':['tanh', 'relu'], 'learning_rate':['constant', 'adaptive'], 'max_iter' :[100, 150]}
#parameters = {'hidden_layer_sizes':[(10,), (15,), (10,5), (20,7,3)]}

mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=15)
mlp_random.fit(X_train1, y_train)
grid_parm=mlp_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
mlp = MLPClassifier(**grid_parm)
mlp.fit(X_train1,y_train)
mlp_predict = mlp.predict(X_test1)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for MultiLayer Perceptron:{0:6f}".format(mlp.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for MultiLayer Perceptron")
print(confusion_matrix(y_test,mlp_predict))
print("=== Classification Report ===")
print(classification_report(y_test,mlp_predict))

#get cross-validation report
mlp_cv_score = cross_val_score(mlp, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(mlp_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - MultiLayer Perceptron: ",mlp_cv_score.mean())


{'max_iter': 150, 'learning_rate': 'constant', 'hidden_layer_sizes': (20, 7, 3), 'activation': 'tanh'}
accuracy Score (training) after hypertuning for MultiLayer Perceptron:0.964854
Confusion Matrix after hypertuning for MultiLayer Perceptron
[[38359  1563]
 [   26  5263]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      0.96      0.98     39922
           1       0.77      1.00      0.87      5289

    accuracy                           0.96     45211
   macro avg       0.89      0.98      0.92     45211
weighted avg       0.97      0.96      0.97     45211

=== All AUC Scores ===
[0.98636792 0.835625   0.7821875  0.83435096 0.86038462 0.85951923
 0.98055288 0.99173077 0.949375   0.80257212]


=== Mean AUC Score ===
Mean AUC Score - MultiLayer Perceptron:  0.888266600145138


In [ ]:
#Construct K-Nearest Neighbor Model
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)

neigh.fit(X_res, y_res)
neigh_predict=neigh.predict(X_test1)
print("Accuracy Score (Testing) for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test1,y_test)))
print("Confusion Matrix for KNeighborsClassifier:")
print(confusion_matrix(y_test,neigh_predict))


Accuracy Score (Testing) for KNeighborsClassifier:0.666523
Confusion Matrix for KNeighborsClassifier:
[[9947 3194]
 [2225  884]]


In [ ]:
predictions = neigh.predict(test_data)
sample_submission = pd.read_csv("/gdrive/MyDrive/CIS508/Homesite/sample_submission 3 (1).csv")
sample_submission['QuoteConversion_Flag'] = predictions
sample_submission.to_csv("/gdrive/MyDrive/CIS508/Homesite/submission.csv",index=False)

In [ ]:
#Hyperparameter tuning done for K-Nearest Neighbor classifier

parameters = {'n_neighbors':[3,5,7,9,11], 'weights':['uniform', 'distance'], 'p':[1,2]}


neigh_random = RandomizedSearchCV(neigh,parameters,n_iter=15)
neigh_random.fit(X_train1, y_train)
grid_parm=neigh_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
neigh = KNeighborsClassifier(**grid_parm)
neigh.fit(X_train1,y_train)
neigh_predict = neigh.predict(X_test1)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for KNeighborsClassifier:{0:6f}".format(neigh.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for KNeighborsClassifier")
print(confusion_matrix(y_test,neigh_predict))
print("=== Classification Report ===")
print(classification_report(y_test,neigh_predict))

#get cross-validation report
neigh_cv_score = cross_val_score(neigh, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(neigh_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNeighborsClassifier: ",neigh_cv_score.mean())


{'weights': 'uniform', 'p': 1, 'n_neighbors': 9}
accuracy Score (training) after hypertuning for KNeighborsClassifier:0.887196
Confusion Matrix after hypertuning for KNeighborsClassifier
[[39152   770]
 [ 4330   959]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       0.90      0.98      0.94     39922
           1       0.55      0.18      0.27      5289

    accuracy                           0.89     45211
   macro avg       0.73      0.58      0.61     45211
weighted avg       0.86      0.89      0.86     45211

=== All AUC Scores ===
[0.78139151 0.77776442 0.70728365 0.77163462 0.80454327 0.83709135
 0.76987981 0.77536058 0.78526442 0.84980769]


=== Mean AUC Score ===
Mean AUC Score - KNeighborsClassifier:  0.7860021317126271


In [ ]:
#Construct Linear Support Vector Machine Model
from sklearn.svm import LinearSVC 
linsvm = LinearSVC(max_iter=300) 
linsvm.fit(X_res, y_res) 
linsvm_predict=linsvm.predict(X_test1) 
print("Accuracy Score (Testing) for Linear SVM Classifier:{0:6f}".format(linsvm.score(X_test1,y_test))) 
print("Confusion Matrix for Linear SVM Classifier:") 
print(confusion_matrix(y_test,linsvm_predict))


Accuracy Score (Testing) for Linear SVM Classifier:0.808800
Confusion Matrix for Linear SVM Classifier:
[[13138     3]
 [ 3104     5]]


In [ ]:
#Construct Support Vector Machine Model
from sklearn.svm import SVC 
svm = SVC(max_iter=500) 
svm.fit(X_res, y_res) 
svm_predict=svm.predict(X_test1) 
print("Accuracy Score (Testing) for SVM Classifier:{0:6f}".format(svm.score(X_test1,y_test))) 
print("Confusion Matrix for SVM Classifier:") 
print(confusion_matrix(y_test,svm_predict))

Accuracy Score (Testing) for SVM Classifier:0.395446
Confusion Matrix for SVM Classifier:
[[4369 8772]
 [1052 2057]]


In [ ]:
#Construct Gradient Boosting model

search_grid={'n_estimators':[5,10,20],'learning_rate':[0.01,.1]}
abc =GradientBoostingClassifier()
abc.fit(X_train1, y_train)
abc_predict=abc.predict(X_test1)
print("accuracy Score (training) for Boosting:{0:6f}".format(abc.score(X_test1,y_test)))
print("Confusion Matrix for boosting:")
print(confusion_matrix(y_test,abc_predict))
abc_random = RandomizedSearchCV(abc,search_grid,n_iter=15)
abc_random.fit(X_train1, y_train)
grid_parm_abc=abc_random.best_params_
print(grid_parm_abc)
abc= GradientBoostingClassifier(**grid_parm_abc)
abc.fit(X_train1,y_train)
abc_predict = abc.predict(X_test1)
print("accuracy Score (training) after hypertuning for Boosting:{0:6f}".format(abc.score(X_test1,y_test)))
print("Confusion Matrix after hypertuning for Boosting:")
print(confusion_matrix(y_test,abc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,abc_predict))
abc_cv_score = cross_val_score(abc, X_train1, y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(abc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Boosting: ",abc_cv_score.mean())

accuracy Score (training) for Boosting:1.000000
Confusion Matrix for boosting:
[[39922     0]
 [    0  5289]]
{'n_estimators': 5, 'learning_rate': 0.1}
accuracy Score (training) after hypertuning for Boosting:1.000000
Confusion Matrix after hypertuning for Boosting:
[[39922     0]
 [    0  5289]]
=== Classification Report ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39922
           1       1.00      1.00      1.00      5289

    accuracy                           1.00     45211
   macro avg       1.00      1.00      1.00     45211
weighted avg       1.00      1.00      1.00     45211

=== All AUC Scores ===
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


=== Mean AUC Score ===
Mean AUC Score - Boosting:  1.0


# ***SMOTE***

In [ ]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE(sampling_strategy=0.5)
X_res, y_res = sm.fit_resample(X_train1, y_train)
print('Resampled dataset shape %s' % Counter(y_res))

___________________________________________________________________
SMOTE

Original dataset shape Counter({'QuoteConversion_Flag': 1})
Resampled dataset shape Counter({'QuoteConversion_Flag': 1})


In [ ]:
y_res.value_counts()

QuoteConversion_Flag
0                       39597
1                       19798
dtype: int64

In [ ]:
y_train.value_counts()

QuoteConversion_Flag
0                       39597
1                        9153
dtype: int64

# ***ENSEMBLE METHODS STACKING***

In [ ]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ KNeighborsClassifier(), MLPClassifier(), SVC(), LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
S_Train, S_Test = stacking(models,                   
                           X_res, y_res, test_data,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [6]

model  0:     [KNeighborsClassifier]
    fold  0:  [0.69748805]
    fold  1:  [0.69802680]
    fold  2:  [0.70280827]
    fold  3:  [0.70204741]
    ----
    MEAN:     [0.70009263] + [0.00235836]
    FULL:     [0.70009260]

model  1:     [MLPClassifier]
    fold  0:  [0.86638831]
    fold  1:  [0.73782746]
    fold  2:  [0.72031787]
    fold  3:  [0.83155981]
    ----
    MEAN:     [0.78902336] + [0.06151453]
    FULL:     [0.78902265]

model  2:     [SVC]
    fold  0:  [0.66671156]
    fold  1:  [0.66664422]
    fold  2:  [0.66664422]
    fold  3:  [0.66668912]
    ----
    MEAN:     [0.66667228] + [0.00002916]
    FULL:     [0.66667228]

model  3:     [LinearSVC]
    fold  

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
estimators = [
    ('rf', rfc),
     ('dt', clf),
     ('mlp', mlp),
     ('KNN', neigh),
     ('LSVM', linsvm)
 ]
stack = StackingClassifier( estimators=estimators, final_estimator=RandomForestClassifier())

In [ ]:
stack.fit(X_res, y_res).score(X_test1, y_test)

0.8856

In [ ]:
pred_ens = stack.predict(test_data)

In [ ]:
sample_submission = pd.read_csv("/gdrive/MyDrive/CIS508/Homesite/sample_submission 3 (1).csv")
sample_submission['QuoteConversion_Flag'] = pred_ens
sample_submission.to_csv("/gdrive/MyDrive/CIS508/Homesite/submission.csv",index=False)

In [ ]:
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(X, y)

In [ ]:
stack.get_feature_names_out()

AttributeError: ignored

In [ ]:
# #Hyperparameter tuning done for decision tree classifier
# solvers = ['newton-cg', 'lbfgs', 'liblinear']
# penalty = ['l2']
# c_values = [100, 10, 1.0, 0.1, 0.01]
# # define grid search
# grid = dict(solver=solvers,penalty=penalty,C=c_values)
parameters={'final_estimator__min_samples_split' : range(10,100,10),'final_estimator__max_depth': range(1,20,2)}
stack_random = RandomizedSearchCV(stack,parameters,n_iter=15)
stack_random.fit(X_res, y_res)
grid_parm=stack_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
estimators = [
    ('rf', rfc),
     ('dt', clf),
     ('mlp', mlp),
     ('KNN', neigh),
     ('LSVM', linsvm)
 ]
stack_hpt = StackingClassifier( estimators=estimators, final_estimator=RandomForestClassifier(**grid_parm))
stack_hpt.fit(X_res,y_res)
pred_ens_hpt = stack_hpt.predict(test_data)

KeyboardInterrupt: ignored

In [ ]:
stack.get_params().keys()

dict_keys(['cv', 'estimators', 'final_estimator__bootstrap', 'final_estimator__ccp_alpha', 'final_estimator__class_weight', 'final_estimator__criterion', 'final_estimator__max_depth', 'final_estimator__max_features', 'final_estimator__max_leaf_nodes', 'final_estimator__max_samples', 'final_estimator__min_impurity_decrease', 'final_estimator__min_samples_leaf', 'final_estimator__min_samples_split', 'final_estimator__min_weight_fraction_leaf', 'final_estimator__n_estimators', 'final_estimator__n_jobs', 'final_estimator__oob_score', 'final_estimator__random_state', 'final_estimator__verbose', 'final_estimator__warm_start', 'final_estimator', 'n_jobs', 'passthrough', 'stack_method', 'verbose', 'rf', 'dt', 'mlp', 'KNN', 'LSVM', 'rf__bootstrap', 'rf__ccp_alpha', 'rf__class_weight', 'rf__criterion', 'rf__max_depth', 'rf__max_features', 'rf__max_leaf_nodes', 'rf__max_samples', 'rf__min_impurity_decrease', 'rf__min_samples_leaf', 'rf__min_samples_split', 'rf__min_weight_fraction_leaf', 'rf__n_e

In [ ]:
stack.get_params()

{'cv': None,
 'estimators': [('rf', RandomForestClassifier()),
  ('dt', DecisionTreeClassifier()),
  ('mlp', MLPClassifier(max_iter=100)),
  ('KNN', KNeighborsClassifier(n_neighbors=3)),
  ('LSVM', LinearSVC(max_iter=300))],
 'final_estimator__bootstrap': True,
 'final_estimator__ccp_alpha': 0.0,
 'final_estimator__class_weight': None,
 'final_estimator__criterion': 'gini',
 'final_estimator__max_depth': None,
 'final_estimator__max_features': 'auto',
 'final_estimator__max_leaf_nodes': None,
 'final_estimator__max_samples': None,
 'final_estimator__min_impurity_decrease': 0.0,
 'final_estimator__min_samples_leaf': 1,
 'final_estimator__min_samples_split': 2,
 'final_estimator__min_weight_fraction_leaf': 0.0,
 'final_estimator__n_estimators': 100,
 'final_estimator__n_jobs': None,
 'final_estimator__oob_score': False,
 'final_estimator__random_state': None,
 'final_estimator__verbose': 0,
 'final_estimator__warm_start': False,
 'final_estimator': RandomForestClassifier(),
 'n_jobs': No

In [ ]:
model = MLPClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))
predictions = model.predict(test_data)
sample_submission = pd.read_csv("/gdrive/MyDrive/CIS508/Homesite/sample_submission 3 (1).csv")
sample_submission['QuoteConversion_Flag'] = predictions
sample_submission.to_csv("/gdrive/MyDrive/CIS508/Homesite/submission.csv",index=False)

Final prediction score for ensemble methods: [0.90295385]


ValueError: ignored

In [ ]:
predictions = dt.predict(X_test_v3)
sample_submission = pd.read_csv("/gdrive/MyDrive/CIS508/Santander/sample_submission (1).csv")
sample_submission['TARGET'] = predictions
sample_submission.to_csv("/gdrive/MyDrive/CIS508/Santander/submission.csv",index=False)

In [ ]:
model = KNeighborsClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score for ensemble methods: [0.89778462]


In [ ]:
model = GradientBoostingClassifier()
    
model = model.fit(S_Train, y_res)
y_pred = model.predict(S_Test)
print('Final prediction score for ensemble methods: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score for ensemble methods: [0.89956923]


In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(S_Test))

pred_Probability.head()

,0,1
0,0.999783,0.000217
1,0.999783,0.000217
2,0.999783,0.000217
3,0.999783,0.000217
4,0.999783,0.000217
